In [0]:
!curl https://colab.chainer.org/install | sh -

!apt-get install graphviz
!pip install 'chaineripy'
!pip install 'chainercv'

import sys
import os
from functools import partial
import shutil
import json

from chainer.links.caffe.caffe_function import CaffeFunction
from chainer.serializers import npz
from chainer.datasets import cifar, split_dataset_random, TransformDataset
from chainer import links as L
from chainer import initializers
from google.colab import drive
import numpy as np


base_dir = './drive/My Drive/study'
sys.path.append(base_dir)
drive.mount('/content/drive')

from resnet.networks.resnet101 import ResNet101
from resnet.networks.resnet152 import ResNet152
from execute import run_train
from transform import transform

# Dataset setup
train_val, test = cifar.get_cifar10(scale=255.)
train_size      = int(len(train_val) * 0.9)
train, valid    = split_dataset_random(train_val, train_size, seed=0)

mean = np.mean([x for x, _ in train], axis=(0, 2, 3))
std  = np.std([x for x, _ in train], axis=(0, 2, 3))
if not os.path.exists("/root/.chainer/dataset/pfnet/chainer/models"):
    os.mkdir("/root/.chainer/dataset/pfnet/chainer/models")

shutil.copyfile('drive/My Drive/study/resnet/caffe/ResNet-101-model.caffemodel', 
                "/root/.chainer/dataset/pfnet/chainer/models/ResNet-101-model.caffemodel")

shutil.copyfile('drive/My Drive/study/resnet/caffe/ResNet-152-model.caffemodel', 
                "/root/.chainer/dataset/pfnet/chainer/models/ResNet-152-model.caffemodel")

params = json.load(open(base_dir + '/params.json', "r"))

In [0]:
transform_train = partial(transform, mean=mean, std=std, output_size=(227, 227), train=True, **params)
transform_valid = partial(transform, mean=mean, std=std, output_size=(227, 227), **params)

# Network setup and run
processed_train = TransformDataset(train, transform_train)
processed_valid = TransformDataset(valid, transform_valid)

model = ResNet101(10, init_param=initializers.HeNormal())
net = L.Classifier(model)

params['finetune_layer'] = [net.predictor.fc.W, 
                            net.predictor.fc.b]
params['freeze_layer'] = ['base/conv1', 'base/res2', 'base/res3']
params['result_dir'] = 'drive/My Drive/study/result'
params['save_trainer_interval'] = 3
params['load_dir'] = 'drive/My Drive/study/result/snapshot_epoch-15'

run_train(processed_train, processed_valid, net, **params)

Now loading caffemodel (usually it may take few minutes)
/usr/local/lib/python3.6/dist-packages/chainer/optimizer.py:524: UserWarning: Hook timing attribute not in ('pre', 'post'), defaulting timing to 'pre'.
  warnings.warn("Hook timing attribute not in ('pre', 'post'), "


epoch       main/loss   main/accuracy  val/main/loss  val/main/accuracy  lr          elapsed_time
1           0.595922    0.79351        0.303014       0.904658           0.001       4112.27       
2           0.243621    0.916452       0.204124       0.931529           0.001       6818.41       
3           0.172463    0.941234       0.207162       0.930135           0.001       9073.24       
4           0.141214    0.952236       0.115568       0.959992           0.001       11359.5       
5           0.114487    0.960976       0.180662       0.946059           0.001       13626.4       
6           0.0993866   0.966616       0.0993343      0.968153           0.001       15894.8       
7           0.0877992   0.970017       0.111061       0.960987           0.001       18185.5       
8           0.0816783   0.972706       0.0987642      0.970342           0.001       20454.3       
9           0.0709109   0.975924       0.0948213      0.967158           0.001       22690.5       
10

In [0]:
transform_train = partial(transform, mean=mean, std=std, output_size=(227, 227), train=True, **params)
transform_valid = partial(transform, mean=mean, std=std, output_size=(227, 227), **params)

# Network setup and run
processed_train = TransformDataset(train, transform_train)
processed_valid = TransformDataset(valid, transform_valid)

model = ResNet101(10, init_param=initializers.HeNormal())
net = L.Classifier(model)

params['finetune_layer'] = [net.predictor.fc.W, 
                            net.predictor.fc.b]
params['freeze_layer'] = ['base/conv1', 'base/res2', 'base/res3']
params['result_dir'] = 'drive/My Drive/study/result'
params['save_trainer_interval'] = 3
params['load_dir'] = 'drive/My Drive/study/result/snapshot_epoch-15'

run_train(processed_train, processed_valid, net, **params)

Now loading caffemodel (usually it may take few minutes)
/usr/local/lib/python3.6/dist-packages/chainer/optimizer.py:524: UserWarning: Hook timing attribute not in ('pre', 'post'), defaulting timing to 'pre'.
  warnings.warn("Hook timing attribute not in ('pre', 'post'), "


epoch       main/loss   main/accuracy  val/main/loss  val/main/accuracy  lr          elapsed_time
15          0.0189406   1              0.0852909      0.974721           0.001       36246         
16          0.047841    0.98383        0.0850537      0.973129           0.001       38407.3       
17          0.0467896   0.983564       0.12861        0.964172           0.001       40553.4       
18          0.0427475   0.985953       0.0894746      0.97492            0.001       42698         
19          0.0422989   0.985931       0.106532       0.971537           0.001       44862.2       
20          0.0399321   0.986931       0.10922        0.969745           0.001       47002.6       
21          0.0350948   0.987829       0.0845399      0.973925           0.0001      49145.8       
22          0.0303279   0.989554       0.0821464      0.976314           0.0001      51308.7       
23          0.0292308   0.989976       0.0794436      0.976513           0.0001      53448.9       
24